In [36]:
import imageio.v2 as imageio
import torch   
import torch.nn as nn              
import torch.nn.functional as F       
import torch.optim as optim            

In [37]:
def load_image(filename, start, end):
    tensor_list=[]
    for i in range(start,end+1):
        img_data=torch.tensor(imageio.imread(f"./img/{filename}{i}.png"))
        new_size = (100, 100)
        resized_img_data = F.interpolate(img_data.unsqueeze(0).permute(0, 3, 1, 2), size=new_size, mode='bilinear', align_corners=False)
        resized_img_data = resized_img_data.squeeze(0).permute(1, 2, 0)
        tensor_list.append(resized_img_data)
    return torch.stack(tensor_list)

In [38]:
sento=load_image("sento",0,19)
why=load_image("why",0,12)
x = torch.cat([sento, why], dim=0) # 사진 33개, shape : [33,100,100,4]
x=x.float()
y = [0 for _ in range(20)]+[1 for _ in range(13)] # 0 : 전투방, 1: 비밀방 
dataset=dict(zip(x,y))
for i in dataset.items():
    print(i)

(tensor([[[ 13.,  71.,  75., 255.],
         [ 11.,  66.,  72., 255.],
         [ 11.,  63.,  68., 255.],
         ...,
         [ 14.,  74.,  71., 255.],
         [ 16.,  77.,  78., 255.],
         [ 16.,  80.,  84., 255.]],

        [[ 11.,  65.,  70., 255.],
         [ 10.,  62.,  68., 255.],
         [  9.,  61.,  66., 255.],
         ...,
         [ 11.,  66.,  67., 255.],
         [ 14.,  71.,  71., 255.],
         [ 16.,  74.,  76., 255.]],

        [[  8.,  63.,  64., 255.],
         [  8.,  61.,  62., 255.],
         [  8.,  58.,  62., 255.],
         ...,
         [  8.,  63.,  64., 255.],
         [  9.,  65.,  68., 255.],
         [ 12.,  69.,  73., 255.]],

        ...,

        [[ 12.,  61.,  64., 255.],
         [ 11.,  59.,  62., 255.],
         [ 10.,  53.,  56., 255.],
         ...,
         [  8.,  27.,  31., 255.],
         [  8.,  28.,  32., 255.],
         [  8.,  30.,  34., 255.]],

        [[ 14.,  70.,  73., 255.],
         [ 13.,  68.,  71., 255.],
         [ 

In [39]:
import torch
import torch.nn as nn
# 256p

model=nn.Sequential(
    nn.Linear(40000, 10000),
    nn.Tanh(),
    nn.Linear(10000,2000),
    nn.Tanh(),
    nn.Linear(2000,500),
    nn.Tanh(),
    nn.Linear(500,2),
    nn.LogSoftmax(dim=1))

optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn=nn.NLLLoss()
for epoch in range(100):
    correct=0
    total=0
    for img,label in dataset.items():
        out=model(img.view(-1).unsqueeze(0))
        loss=loss_fn(out,torch.tensor([label]))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(out, 1)
        total += 1
        correct += (predicted == label).sum().item()
    print(f"Epoch: {epoch}, Loss: {float(loss)}, Accuracy : {correct/total}")

Epoch: 0, Loss: 0.008482498116791248, Accuracy : 0.9393939393939394
Epoch: 1, Loss: 0.015220098197460175, Accuracy : 0.9393939393939394
Epoch: 2, Loss: 0.01663750596344471, Accuracy : 0.9393939393939394
Epoch: 3, Loss: 0.015400656498968601, Accuracy : 0.9393939393939394
Epoch: 4, Loss: 0.011907803826034069, Accuracy : 0.9393939393939394
Epoch: 5, Loss: 0.00933071319013834, Accuracy : 0.9393939393939394
Epoch: 6, Loss: 0.006485604215413332, Accuracy : 0.9393939393939394
Epoch: 7, Loss: 0.0038600473199039698, Accuracy : 1.0
Epoch: 8, Loss: 0.002173920162022114, Accuracy : 1.0
Epoch: 9, Loss: 0.001453534816391766, Accuracy : 1.0
Epoch: 10, Loss: 0.0011755467858165503, Accuracy : 1.0
Epoch: 11, Loss: 0.0008735416340641677, Accuracy : 1.0
Epoch: 12, Loss: 0.0007406114018522203, Accuracy : 1.0
Epoch: 13, Loss: 0.000608854868914932, Accuracy : 1.0
Epoch: 14, Loss: 0.000516757951118052, Accuracy : 1.0
Epoch: 15, Loss: 0.00044312194222584367, Accuracy : 1.0
Epoch: 16, Loss: 0.000416192226111888

In [50]:
 # 0 : 전투방, 1: 비밀방 
 # 예측하기
torch_layer=[]
for i in list(model.parameters()):
    torch_layer.append(i)


In [49]:
img_data=torch.tensor(imageio.imread(f"./img/sento3.png"))
new_size = (100, 100)
resized_img_data = F.interpolate(img_data.unsqueeze(0).permute(0, 3, 1, 2), size=new_size, mode='bilinear', align_corners=False)
resized_img_data = resized_img_data.squeeze(0).permute(1, 2, 0)
resized_img_data.shape

torch.Size([100, 100, 4])

In [57]:
x = resized_img_data.view(-1, 40000)  # 이미지 데이터를 1차원으로 평탄화
x = x.float()
x = F.linear(x, weight=torch_layer[0], bias=torch_layer[1])  # 첫 번째 선형 레이어
x = F.tanh(x)  # Tanh 활성화 함수
x = F.linear(x, weight=torch_layer[2], bias=torch_layer[3])  # 두 번째 선형 레이어
x = F.tanh(x)  # Tanh 활성화 함수
x = F.linear(x, weight=torch_layer[4], bias=torch_layer[5])  # 세 번째 선형 레이어
x = F.tanh(x)  # Tanh 활성화 함수
x = F.linear(x, weight=torch_layer[6], bias=torch_layer[7])  # 출력 레이어

# 출력 결과
_, predicted = torch.max(x, 1)
print(predicted) # 0이 나와서 전투방인 걸 제대로 인식했다!

tensor([0])


In [ ]:
# 아~너무 힘들다~쓰러지고 싶어!